## Loading and Inspecting Data

In [19]:
import json
import pandas as pd
import numpy as np
import os, sys
import yaml
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import src.data_management as my_dm
seed=456

In [20]:


with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

train_file_path = config["data"]["input_train_path"]
test_file_path = config["data"]["input_test_path"]

train_data = []

print(f"Loading data from '{train_file_path}'...")
try:
    with open(train_file_path, 'r') as f:
        for line in f:
            # json.loads() parses one line (one JSON object) into a Python dictionary
            train_data.append(json.loads(line))

    print(f"Successfully loaded {len(train_data)} battles.")

except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{train_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

Loading data from '../data/raw/train.jsonl'...
Successfully loaded 10000 battles.


In [28]:
def create_simple_features(data: list[dict]) -> pd.DataFrame:
    """
    A very basic feature extraction function.
    It only uses the aggregated base stats of the player's team and opponent's lead.
    """
    feature_list = []
    for battle in data:
        features = {}
        features["battle_id"] = battle.get("battle_id", -1)
        if battle.get('player_won') is not None:
            features['player_won'] = int(battle['player_won'])
        # --- Player 1 Team Features ---
        p1_team = battle.get('p1_team_details', [])
        if p1_team:
            """
            stats = ['base_hp', 'base_atk', 'base_def', 'base_spa', 'base_spd', 'base_spe']
            for stat in stats:               ### This helps the model to have a better idea abt the time instead of having only mean
                values = [p.get(stat, 0) for p in p1_team]
                features[f'p1_mean_{stat}'] = np.mean(values)
                features[f'p1_min_{stat}'] = np.min(values)
                features[f'p1_max_{stat}'] = np.max(values)
                features[f'p1_std_{stat}'] = np.std(values)
            features['p1_mean_hp'] = np.mean([p.get('base_hp', 0) for p in p1_team])
            features['p1_mean_spe'] = np.mean([p.get('base_spe', 0) for p in p1_team])
            features['p1_mean_atk'] = np.mean([p.get('base_atk', 0) for p in p1_team])
            features['p1_mean_def'] = np.mean([p.get('base_def', 0) for p in p1_team])
            features['p1_mean_spa'] = np.mean([p.get('base_spa', 0) for p in p1_team])
            features['p1_mean_spd'] = np.mean([p.get('base_spd', 0) for p in p1_team])
            features['p1_mean_stats'] = np.mean([features['p1_mean_hp'], features['p1_mean_spe'], features['p1_mean_atk'], features['p1_mean_def'], features['p1_mean_spa'], features['p1_mean_spd']])
            ### We can also build derivated feature like how much is off/def our team
            # team stats
            base_atk = features['p1_mean_base_atk']
            base_spa = features['p1_mean_base_spa']
            base_def = features['p1_mean_base_def']
            base_spd = features['p1_mean_base_spd']
            base_spe = features['p1_mean_base_spe']
            base_hp  = features['p1_mean_base_hp']

            ## constructing new features
            offense = base_atk + base_spa
            defense = base_def + base_spd
            features['p1_offense_mean']    = offense
            features['p1_defense_mean']    = defense
            features['p1_atk_def_ratio']   = offense / (defense + 1e-6)
            # average per-Pokémon total base stats
            p1_totals = [sum(p.get(s, 0) for s in stats) for p in p1_team]
            features['p1_total_base_power'] = float(np.mean(p1_totals))
            features['p1_stat_variety']     = float(np.std(p1_totals))
            features['p1_style_index']      = offense / (offense + defense + 1e-6)
            features['p1_hp_ratio']         = base_hp / (offense + defense + base_spe + 1e-6)
            # fastest member speed (robust comparator vs P2 lead)
            features['p1_max_speed']        = float(np.max([p.get('base_spe', 0) for p in p1_team]))
        
    
            """
            """
###########################################AGGIUNTA####################################################
          # Estrazione tipi
            type_counts = {t: 0 for t in all_types}
            for p in p1_team:
                for t in p.get('types', []):
                    type_counts[t] += 1
            team_size = len(p1_team)
            for t in all_types:
                features[f'p1_type_{t}'] = type_counts[t] / team_size if team_size > 0 else 0
##########################################AGGIUNTA#####################################################
        
        # --- Player 2 Lead Features ---
        p2_lead = battle.get('p2_lead_details')
        if p2_lead:
            # Player 2's lead Pokémon's stats
            features['p2_lead_hp'] = p2_lead.get('base_hp', 0)
            features['p2_lead_spe'] = p2_lead.get('base_spe', 0)
            features['p2_lead_atk'] = p2_lead.get('base_atk', 0)
            features['p2_lead_def'] = p2_lead.get('base_def', 0)
            features['p2_lead_spd'] = p2_lead.get('base_spd', 0)
            features['p2_lead_spa'] = p2_lead.get('base_spa', 0)
            ## types of p2 lead
            for t in all_types:
                features[f'p2_lead_type_{t}'] = 0.0
            for t in p2_lead.get('types', []):
                if t in all_types:
                    features[f'p2_lead_type_{t}'] = 1.0
                
            """
        ## Extracting battle features
        timeline = battle.get("battle_timeline", [])
        if len(timeline) > 0:
            """
            # Average HP percentage for both players
            p1_hp = [turn["p1_pokemon_state"].get("hp_pct", np.nan) for turn in timeline]
            p2_hp = [turn["p2_pokemon_state"].get("hp_pct", np.nan) for turn in timeline]

            features["p1_mean_hp_pct"] = np.nanmean(p1_hp)
            features["p2_mean_hp_pct"] = np.nanmean(p2_hp)
            features["p1_final_hp"] = p1_hp[-1]
            features["p2_final_hp"] = p2_hp[-1]
            features["p1_total_damage"] = max(p1_hp) - min(p1_hp)
            features["p2_total_damage"] = max(p2_hp) - min(p2_hp)

            # Count total moves used
            p1_moves = [turn.get("p1_move_details", {}).get("name") for turn in timeline if turn.get("p1_move_details")]
            p2_moves = [turn.get("p2_move_details", {}).get("name") for turn in timeline if turn.get("p2_move_details")]
            features["p1_total_moves"] = len(p1_moves)
            features["p2_total_moves"] = len(p2_moves)

            # Count unique move types used
            p1_move_types = [turn["p1_move_details"].get("type") for turn in timeline if turn.get("p1_move_details")]
            p2_move_types = [turn["p2_move_details"].get("type") for turn in timeline if turn.get("p2_move_details")]
            features["p1_unique_move_types"] = len(set(p1_move_types))
            features["p2_unique_move_types"] = len(set(p2_move_types))
            """

            p1_pkmns =  set([turn["p1_pokemon_state"]["name"] for turn in timeline])
            p2_pkmns = set([turn["p2_pokemon_state"]["name"] for turn in timeline])

            ### status and effects features
            total_statuses = ['slp', 'fnt', 'tox', 'psn', 'brn', 'frz', 'par', 'nostatus']
            p1_status = [turn["p1_pokemon_state"].get("status") for turn in timeline]
            p2_status = [turn["p2_pokemon_state"].get("status") for turn in timeline]

            total_effects = ['disable', 'firespin', 'confusion', 'substitute', 'wrap', 'clamp', 'typechange', 'reflect', 'noeffect']
            p1_effects = [turn["p1_pokemon_state"].get("effects") for turn in timeline]
            p1_effects = [effect for effects in p1_effects for effect in effects]
            p2_effects = [turn["p2_pokemon_state"].get("effects") for turn in timeline]
            p2_effects = [effect for effects in p2_effects for effect in effects]

            ## general status features

            # Count total statuses inflicted

            features["p1_status_changes"] = sum(1 for s in p1_status if s not in ["nostatus", "noeffect", None])
            features["p2_status_changes"] = sum(1 for s in p2_status if s not in ["nostatus", "noeffect", None])

            # Count single status count 

            dict_status_p1 = {status : 0 for status in total_statuses}
            dict_status_p2 = {status : 0 for status in total_statuses}
            for turn in timeline:
                if turn["p1_pokemon_state"].get("status"):
                    turn_status = turn["p1_pokemon_state"].get("status")
                    dict_status_p1[turn_status] += 1
                if turn["p2_pokemon_state"].get("status"):
                    turn_status = turn["p2_pokemon_state"].get("status")
                    dict_status_p2[turn_status] += 1
            
            for status in total_statuses:
                features[f"p1_{status}_count"] = dict_status_p1.get(status, 0)
                features[f"p2_{status}_count"] = dict_status_p2.get(status, 0)

            ## fnt features
            
            # Difference in fnt pkmn

            features["p2-p1_fnt_pokemon_number"] = features["p1_fnt_count"] - features["p2_fnt_count"]

            # fnt pkmn over total pokemon

            features["p1_fnt_over_total_pkmn"] = features["p1_fnt_count"]/len(p1_pkmns)
            features["p2_fnt_over_total_pkmn"] = features["p2_fnt_count"]/len(p2_pkmns)

            ## effects features

            # Count turns with effects like "reflect", "light screen", etc.

            dict_effects_p1 = {effect : 0 for effect in total_effects}
            dict_effects_p2 = {effect : 0 for effect in total_effects}
            for turn in timeline:
                if turn["p1_pokemon_state"].get("effects"):
                    turn_effects_p1 = turn["p1_pokemon_state"].get("effects")
                    for effect in turn_effects_p1:
                        dict_effects_p1[effect] += 1
                if turn["p2_pokemon_state"].get("effects"):
                    turn_effects_p2 = turn["p2_pokemon_state"].get("effects")
                    for effect in turn_effects_p2:
                        dict_effects_p2[effect] += 1            
            for effect in total_effects:
                features[f"p1_{effect}_count"] = dict_effects_p1.get(effect, 0)
                features[f"p2_{effect}_count"] = dict_effects_p2.get(effect, 0)

            # Boosts (attack, defense, etc.)
            #boost_keys = ["atk", "def", "spa", "spd", "spe"]
            #for key in boost_keys:
            #    p1_boosts = [turn["p1_pokemon_state"]["boosts"].get(key, 0) for turn in timeline]
            #    p2_boosts = [turn["p2_pokemon_state"]["boosts"].get(key, 0) for turn in timeline]
            #    features[f"p1_mean_boost_{key}"] = np.mean(p1_boosts)
            #    features[f"p2_mean_boost_{key}"] = np.mean(p2_boosts)

            # Number of time the player switched pokemon
            features["p1_switch_number"] = sum([1 for turn in timeline if turn["p1_move_details"] == None])
            features["p2_switch_number"] = sum([1 for turn in timeline if turn["p2_move_details"] == None])

            # Number of SPECIAL or PHYSICAL moves and Number of STATUS moves of p1 and p2
            features["p1_attack_moves"] = sum([1 for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
            features["p2_attack_moves"] = sum([1 for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
            features["p1_status_moves"] = sum([1 for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"]["category"] not in ["SPECIAL", "PHYSICAL"]])
            features["p2_status_moves"] = sum([1 for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"]["category"] not in ["SPECIAL", "PHYSICAL"]])

            # Number of same pokemon type moves
            features["p1_same_type_moves_number"] = sum([1 for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"]["type"] in my_dm.pokemon_type(turn["p1_pokemon_state"]["name"]) and turn["p1_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
            features["p2_same_type_moves_number"] = sum([1 for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"]["type"] in my_dm.pokemon_type(turn["p2_pokemon_state"]["name"]) and turn["p2_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])

            # Average of multiplier effectivness
            features["p1_effectivness_avg"] = np.nanmean([my_dm.move_effectiveness(turn["p1_move_details"]["type"], turn["p2_pokemon_state"]["name"]) for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
            features["p2_effectivness_avg"] = np.nanmean([my_dm.move_effectiveness(turn["p2_move_details"]["type"], turn["p1_pokemon_state"]["name"]) for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])

            # Number of supereffective moves
            features["p1_supereffective_moves_count"] = sum([my_dm.is_supereffective(my_dm.move_effectiveness(turn["p1_move_details"]["type"], turn["p2_pokemon_state"]["name"])) for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
            features["p2_supereffective_moves_count"] = sum([my_dm.is_supereffective(my_dm.move_effectiveness(turn["p2_move_details"]["type"], turn["p1_pokemon_state"]["name"])) for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
            
            # Sum of priority moves
            features["p1_priority_moves"] = sum([1 for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"].get("priority")])
            features["p2_priority_moves"] = sum([1 for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"].get("priority")])

            # Number of supereffective pokemon of p1 in respect to p2 

            features["p1_supereffective_density"] = sum(1 for pkmn1 in p1_pkmns for pkmn2 in p2_pkmns if my_dm.is_supereffective(my_dm.pkmn_effectiveness(pkmn1, pkmn2)))/(len(p1_pkmns)*len(p2_pkmns))
            features["p2_supereffective_density"] = sum(1 for pkmn1 in p1_pkmns for pkmn2 in p2_pkmns if my_dm.is_supereffective(my_dm.pkmn_effectiveness(pkmn2, pkmn1)))/(len(p1_pkmns)*len(p2_pkmns))
            features["p1-p2_se_densities"] = features["p2_supereffective_density"] - features["p1_supereffective_density"]

            # P1 pkmn having at least a supereffective target in p2 team
            features["p1_attackers_share"] = sum(any(my_dm.is_supereffective(my_dm.pkmn_effectiveness(pkmn1, pkmn2)) for pkmn2 in p2_pkmns) for pkmn1 in p1_pkmns) / len(p1_pkmns)

            # P1 pkmn that are target of at least one pkmn in team 2
            features["p1_defensive_share"] = sum(any(my_dm.is_supereffective(my_dm.pkmn_effectiveness(pkmn2, pkmn1)) for pkmn2 in p2_pkmns) for pkmn1 in p1_pkmns) / len(p2_pkmns)

            # mean of hp percentage for p1 team and p2 team on last informations
            p1_hp_pctg = {p1_pkmn : None for p1_pkmn in p1_pkmns}
            p2_hp_pctg = {p2_pkmn : None for p2_pkmn in p2_pkmns}
            for turn in timeline:
                p1_hp_pctg.update({turn["p1_pokemon_state"]["name"] : turn["p1_pokemon_state"]["hp_pct"]})
                p2_hp_pctg.update({turn["p2_pokemon_state"]["name"] : turn["p2_pokemon_state"]["hp_pct"]})
            
            features["p1_remain_health_avg"] = sum(p1_hp_pctg.values())/len(p1_pkmns)
            features["p2_remain_health_avg"] = sum(p2_hp_pctg.values())/len(p2_pkmns)
            features["health_difference"] = features["p2_remain_health_avg"] - features["p1_remain_health_avg"]


        feature_list.append(features)
        
    return pd.DataFrame(feature_list).fillna(0)



In [17]:
p1_effects = []
p2_effects = []
for battle in train_data:
    timeline = battle["battle_timeline"]
    p1_effects.extend([turn["p1_pokemon_state"].get("effects") for turn in timeline])
    p2_effects.extend([turn["p2_pokemon_state"].get("effects") for turn in timeline])
total_effects = []
for p in p1_effects:
    
    for p1 in p:
        total_effects.append(p1)
print(set(total_effects))

{'firespin', 'confusion', 'substitute', 'wrap', 'clamp', 'typechange', 'reflect', 'noeffect'}


In [29]:

# Create feature DataFrames for both training and test sets
print("Processing training data...")
train_df = create_simple_features(train_data)
train_df = train_df.sample(frac=1, random_state=seed).reset_index(drop=True)

### removing correlations
corr_matrix = train_df.corr().abs()
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)
threshold = 0.9
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print("Variabili da rimuovere:", to_drop)
train_df = train_df.drop(columns=to_drop)
## exporting in csv
train_df.to_csv(config["data"]["processed_train_path"], index=False)

print("\nProcessing test data...")
test_data = []
with open(test_file_path, 'r') as f:
    for line in f:
        test_data.append(json.loads(line))
test_df = create_simple_features(test_data)
test_df =test_df.sample(frac=1, random_state=seed).reset_index(drop=True)
## removing correlations in test
test_df = test_df.drop(columns=to_drop)


test_df.to_csv(config["data"]["processed_test_path"], index=False)

Processing training data...


C:\Users\Edoardo\AppData\Local\Temp\ipykernel_21160\4105063376.py:205: RuntimeWarning: Mean of empty slice
  features["p2_effectivness_avg"] = np.nanmean([my_dm.move_effectiveness(turn["p2_move_details"]["type"], turn["p1_pokemon_state"]["name"]) for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
C:\Users\Edoardo\AppData\Local\Temp\ipykernel_21160\4105063376.py:204: RuntimeWarning: Mean of empty slice
  features["p1_effectivness_avg"] = np.nanmean([my_dm.move_effectiveness(turn["p1_move_details"]["type"], turn["p2_pokemon_state"]["name"]) for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])


Variabili da rimuovere: ['p1_nostatus_count', 'p2_nostatus_count', 'p2-p1_fnt_pokemon_number', 'p1_fnt_over_total_pkmn', 'p2_fnt_over_total_pkmn', 'p2_typechange_count', 'p1_noeffect_count', 'p2_noeffect_count']

Processing test data...


C:\Users\Edoardo\AppData\Local\Temp\ipykernel_21160\4105063376.py:204: RuntimeWarning: Mean of empty slice
  features["p1_effectivness_avg"] = np.nanmean([my_dm.move_effectiveness(turn["p1_move_details"]["type"], turn["p2_pokemon_state"]["name"]) for turn in timeline if turn.get("p1_move_details") and turn["p1_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])
C:\Users\Edoardo\AppData\Local\Temp\ipykernel_21160\4105063376.py:205: RuntimeWarning: Mean of empty slice
  features["p2_effectivness_avg"] = np.nanmean([my_dm.move_effectiveness(turn["p2_move_details"]["type"], turn["p1_pokemon_state"]["name"]) for turn in timeline if turn.get("p2_move_details") and turn["p2_move_details"]["category"] in ["SPECIAL", "PHYSICAL"]])


In [5]:
print(my_dm.pokemon_type("starmie"), my_dm.pokemon_type("exeggutor"))

['WATER', 'PSYCHIC'] ['GRASS', 'PSYCHIC']


In [18]:
for battle in train_data:

    p1_pkmns =  set([turn["p1_pokemon_state"]["name"] for turn in battle["battle_timeline"]])
    p2_pkmns = set([turn["p2_pokemon_state"]["name"] for turn in battle["battle_timeline"]])
    a = sum(1 for pkmn1 in p1_pkmns for pkmn2 in p2_pkmns if my_dm.is_supereffective(my_dm.pkmn_effectiveness(pkmn2, pkmn1)))/(len(p1_pkmns)*len(p2_pkmns))
    #a = [[pkmn1, pkmn2, my_dm.is_supereffective(my_dm.pkmn_effectiveness(pkmn2, pkmn2))] for pkmn1 in p1_pkmns for pkmn2 in p2_pkmns]
    print(a)

0.0
0.0
0.08333333333333333
0.0
0.0
0.06666666666666667
0.1388888888888889
0.05
0.0
0.027777777777777776
0.06666666666666667
0.04
0.06666666666666667
0.13333333333333333
0.1875
0.125
0.04
0.1
0.05555555555555555
0.08333333333333333
0.0
0.0
0.0
0.0
0.08333333333333333
0.1
0.1
0.0625
0.06666666666666667
0.05555555555555555
0.0
0.08
0.0
0.19444444444444445
0.027777777777777776
0.0
0.05555555555555555
0.03333333333333333
0.1111111111111111
0.041666666666666664
0.06666666666666667
0.041666666666666664
0.08
0.0
0.08
0.0
0.1111111111111111
0.1
0.0625
0.06666666666666667
0.041666666666666664
0.05
0.08333333333333333
0.04
0.08333333333333333
0.16
0.0
0.027777777777777776
0.03333333333333333
0.1
0.05
0.0
0.06666666666666667
0.1111111111111111
0.16666666666666666
0.1
0.16666666666666666
0.12
0.0
0.0
0.1
0.15
0.1388888888888889
0.15
0.1
0.2
0.13333333333333333
0.08333333333333333
0.08333333333333333
0.06666666666666667
0.13333333333333333
0.1
0.05
0.16
0.125
0.25
0.027777777777777776
0.05
0.0
0.0
